# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

In [1]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


## Serve Single REST Model

In [9]:
!helm install mymodel ../helm-charts/seldon-single-model

NAME: mymodel
LAST DEPLOYED: Sat Jan  4 17:08:31 2020
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [3]:
!helm template ../helm-charts/seldon-single-model | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": machinelearning.seldon.io/v1,
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {"app":"seldon"},
        "name": "RELEASE-NAME"
    },
    "spec": {
        "annotations":{"seldon.io/engine-log-message-type":"seldon.message.pair","seldon.io/engine-log-messages-externally":"false","seldon.io/engine-log-requests":"false","seldon.io/engine-log-responses":"false","seldon.io/headless-svc":"false"},
        "name": "RELEASE-NAME",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_rest:1.3",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {"requests":{"memory":"1Mi"}},
                           

In [10]:
!kubectl rollout status deploy/mymodel-mymodel-de240ba

Waiting for deployment "mymodel-mymodel-de240ba" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-de240ba" successfully rolled out


### Get predictions

In [11]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [12]:
r = sc.predict(transport="rest")
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.6042750134087251
  }
}

Response:
meta {
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0901018762156463
  }
}



In [13]:
!helm delete mymodel

release "mymodel" uninstalled


## Serve Single GRPC Model

In [ ]:
!helm install mymodel ../helm-charts/seldon-single-model --set protocol=GRPC

In [ ]:
!helm template ../helm-charts/seldon-single-model --set protocol=GRPC | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-2a00e84

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador", debug=True)

#### REST Request

In [ ]:
r = sc.predict(transport="grpc")
print(r)

In [ ]:
!helm delete mymodel

## Serve REST AB Test

In [ ]:
!helm install myabtest ../helm-charts/seldon-abtest

In [ ]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-0cce7b2
!kubectl rollout status deploy/myabtest-myabtest-ba661ba

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [ ]:
r = sc.predict(transport="rest")
print(r)

In [ ]:
!helm delete myabtest

## Serve GRPC AB Test

In [ ]:
!helm install myabtest ../helm-charts/seldon-abtest --set protocol=GRPC

In [ ]:
!helm template ../helm-charts/seldon-abtest --set protocol=GRPC | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-050eba7
!kubectl rollout status deploy/myabtest-myabtest-5ffb9f0

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [ ]:
r = sc.predict(transport="grpc")
print(r)

In [ ]:
!helm delete myabtest

## Serve REST Multi-Armed Bandit

In [ ]:
!helm install mymab ../helm-charts/seldon-mab

In [ ]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymab-mymab-0cce7b2
!kubectl rollout status deploy/mymab-mymab-b8038b2
!kubectl rollout status deploy/mymab-mymab-ba661ba 

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [ ]:
r = sc.predict(transport="rest")
print(r)

In [ ]:
!helm delete mymab

## Serve GRPC Multi-Armed Bandit

In [ ]:
!helm install mymab ../helm-charts/seldon-mab --set protocol=GRPC

In [ ]:
!helm template ../helm-charts/seldon-mab --set protocol=GRPC | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymab-mymab-050eba7
!kubectl rollout status deploy/mymab-mymab-5f10a24
!kubectl rollout status deploy/mymab-mymab-5ffb9f0 

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [ ]:
r = sc.predict(transport="grpc")
print(r)

In [ ]:
!helm delete mymab